### PyTorch

In [1]:
## Imports (and set seeds for reproducibility)
import os
import random
import torch
import torch.nn as nn
import numpy as np

def fix_seeds(seed=75):
  SEED = seed
  random.seed(seed)
  os.environ['PYTHONHASHSEED'] = str(SEED)
  np.random.seed(SEED)
  torch.manual_seed(SEED)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False
  if (torch.cuda.is_available()):
    torch.cuda.manual_seed(SEED)

fix_seeds()

In [2]:
## User inputs

# shape = (1,3,2,2)
input = torch.tensor([[[[0.4963, 0.7682],
                        [0.0885, 0.1320]],

                       [[0.3074, 0.6341],
                        [0.4901, 0.8964]],

                       [[0.4556, 0.6323],
                        [0.3489, 0.4017]]]])
input.requires_grad = True
prob = 0.2

In [3]:
print('INPUT for Spatial Dropout: \n')
print(input)

INPUT for Spatial Dropout: 

tensor([[[[0.4963, 0.7682],
          [0.0885, 0.1320]],

         [[0.3074, 0.6341],
          [0.4901, 0.8964]],

         [[0.4556, 0.6323],
          [0.3489, 0.4017]]]], requires_grad=True)


In [4]:
## Forward()
spatial_dropout = nn.Dropout2d(prob)

In [5]:
out = spatial_dropout(input)

In [6]:
## this is the output from Forward() of Spatial Dropout
print('OUTPUT from Spatial Dropout: \n')
print(out)

OUTPUT from Spatial Dropout: 

tensor([[[[0.6204, 0.9602],
          [0.1106, 0.1650]],

         [[0.0000, 0.0000],
          [0.0000, 0.0000]],

         [[0.5695, 0.7904],
          [0.4361, 0.5021]]]], grad_fn=<MulBackward0>)


In [7]:
## this is gy for Spatial Dropout layer simulated as a tensor filled with an arbitrary sequence of values.
a = torch.numel(out) + 1
a = float(a)
gy = torch.arange(1.,a).reshape(out.shape)
print('Hypothetical gy for Spatial Dropout: \n')
print(gy)

Hypothetical gy for Spatial Dropout: 

tensor([[[[ 1.,  2.],
          [ 3.,  4.]],

         [[ 5.,  6.],
          [ 7.,  8.]],

         [[ 9., 10.],
          [11., 12.]]]])


In [8]:
## Backward()
out.backward(gy)

In [9]:
## this is g for Spatial Dropout layer
g = input.grad
print('g for Spatial Dropout: \n')
print(g)

g for Spatial Dropout: 

tensor([[[[ 1.2500,  2.5000],
          [ 3.7500,  5.0000]],

         [[ 0.0000,  0.0000],
          [ 0.0000,  0.0000]],

         [[11.2500, 12.5000],
          [13.7500, 15.0000]]]])


### mlpack

Equivalent code to reproduce same results in mlpack :
```cpp
#include <iostream>
#include <mlpack/methods/ann/dists/bernoulli_distribution.hpp>
#include <armadillo>
#define PRINT 1

using namespace mlpack::ann;
using namespace std;
using namespace arma;

int main()
{
  // Using this only for demo purposes. This will dropout second channel always.
  arma_rng::set_seed(0);

  // User input
  arma::mat input(12, 1);
  input << 0.4963 << 0.0885 << 0.7682 << 0.1320 << 0.3074 << 0.4901 << 0.6341 << 0.8964 << 0.4556 << 0.3489 << 0.6323 << 0.4017 << endr;
  input = input.t();

  double ratio = 0.2;
  size_t size = 3; // input channels

  arma::mat output;
  output.zeros(arma::size(input));

  // Forward()
  size_t batchSize = input.n_cols;
  size_t inputSize = input.n_rows / size;
  double scale = 1.0 / (1.0 - ratio);
  arma::cube inputTemp(const_cast<arma::mat&>(input).memptr(), inputSize, size, batchSize, false, false);
  arma::cube outputTemp(const_cast<arma::mat&>(output).memptr(), inputSize, size, batchSize, false, false);
  arma::mat probabilities(1, size);
  arma::mat maskRow(1, size);
  arma::mat mask;
  probabilities.fill(ratio);
  BernoulliDistribution<> bernoulli_dist(probabilities, false);
  maskRow = bernoulli_dist.Sample();
  mask = arma::repmat(maskRow, inputSize, 1);

  for(size_t n = 0; n < batchSize; n++)
  {
    arma::mat& inputImage = inputTemp.slice(n);
    arma::mat& outputImage = outputTemp.slice(n);
    outputImage = inputImage % mask * scale;

    if(PRINT)
    {
      cout << "Image " << n << " calculations: " << endl;
      cout << "-----------------------------------" << endl;
      cout << "-----------------------------------" << endl;
      cout << "INPUT for Spatial Dropout: " << endl;
      cout << inputImage << endl;
      cout << "-----------------------------------" << endl;
      cout << "OUTPUT for Spatial Dropout: " << endl;
      cout << outputImage << endl;
      cout << "-----------------------------------" << endl;
    }
  }

  // this is gy for Spatial Dropout layer simulated as a tensor filled with an arbitrary sequence of values.
  arma::mat gy;
  gy << 1 << 3 << 2 << 4 << 5 << 7 << 6 << 8 << 9 << 11 << 10 << 12 << endr;
  gy = gy.t();

  // Backward()
  arma::mat g;
  g.zeros(arma::size(input));

  arma::cube gyTemp(const_cast<arma::mat&>(gy).memptr(), inputSize, size, batchSize, false, false);
  arma::cube gTemp(const_cast<arma::mat&>(g).memptr(), inputSize, size, batchSize, false, false);

  for(size_t n = 0; n < batchSize; n++)
  {
    arma::mat& gyImage = gyTemp.slice(n);
    arma::mat& gImage = gTemp.slice(n);

    gImage = gyImage % mask * scale;

    if(PRINT)
    {
      cout << "Image " << n << " calculations: " << endl;
      cout << "-----------------------------------" << endl;
      cout << "-----------------------------------" << endl;
      cout << "Hypothetical gy for Spatial Dropout: " << endl;
      cout << gyImage << endl;
      cout << "-----------------------------------" << endl;
      cout << "g for Spatial Dropout: " << endl;
      cout << gImage << endl;
      cout << "-----------------------------------" << endl;
    }
  }

  return 0;
}

```

Command for running the above cpp file locally on a Mac :

```bash
g++ -std=c++11 -stdlib=libc++ test.cpp -o test -larmadillo -lmlpack && ./test
```

mlpack output :
```md
Image 0 calculations: 
-----------------------------------
-----------------------------------
INPUT for Spatial Dropout: 
   0.4963   0.3074   0.4556
   0.0885   0.4901   0.3489
   0.7682   0.6341   0.6323
   0.1320   0.8964   0.4017

-----------------------------------
OUTPUT for Spatial Dropout: 
   0.6204        0   0.5695
   0.1106        0   0.4361
   0.9603        0   0.7904
   0.1650        0   0.5021

-----------------------------------
Image 0 calculations: 
-----------------------------------
-----------------------------------
Hypothetical gy for Spatial Dropout: 
    1.0000    5.0000    9.0000
    3.0000    7.0000   11.0000
    2.0000    6.0000   10.0000
    4.0000    8.0000   12.0000

-----------------------------------
g for Spatial Dropout: 
    1.2500         0   11.2500
    3.7500         0   13.7500
    2.5000         0   12.5000
    5.0000         0   15.0000

-----------------------------------
```